## Main Script

In [1]:
import re
import json
import pandas as pd
import pdfplumber
from collections import defaultdict
from pathlib import Path


def extract_period_from_text(text):
    """Extracts the statement period (e.g., '2025-June') from the first page text."""
    match = re.search(r"Period:\s+([A-Z]+)\s+-\s+(\d{4})", text)
    if match:
        month = match.group(1).capitalize()
        year = match.group(2)
        return f"{year}-{month}"
    return None


def clean_header(header):
    # Removes \n, trims spaces, and joins words cleanly
    return [col.replace("\n", " ").strip() for col in header]


def process_single_statement(file_path):
    """
    Parses a single statement PDF, sets correct data types, and extracts the period.
    Returns the period string and a dictionary of DataFrames.
    """
    TABLE_NAMES = ["Holdings", "Income", "Fees", "Transaction", "Deposit & Withdrawals"]
    NUMERIC_COLS = [
        "Net Amt",
        "Amount",
        "Market Price",
        "Market Value",
        "Cost Price",
        "Unrealized",
        "TD Cost Basis",
        "Commission",
        "Price",
        "Quantity",
    ]
    DATE_COLS = ["Trade Date"]  # Add any other date columns here

    data_collector = defaultdict(list)
    current_table_name = None
    statement_period = None

    with pdfplumber.open(file_path) as pdf:
        # Extract period from the first page
        first_page_text = pdf.pages[0].extract_text()
        statement_period = extract_period_from_text(first_page_text)

        for page in pdf.pages:
            for table in page.extract_tables():
                for row in table:
                    if not any(cell for cell in row if cell and cell.strip()):
                        continue
                    first_cell = row[0].strip() if row[0] else ""
                    if first_cell in TABLE_NAMES:
                        current_table_name = first_cell
                        continue
                    if current_table_name:
                        if "No record found." in first_cell:
                            current_table_name = None
                            continue
                        data_collector[current_table_name].append(row)

    final_dfs = {}
    for name, rows in data_collector.items():
        if not rows:
            continue
        header = clean_header(rows[0])
        data = [row for row in rows[1:] if row != header]
        if not data:
            continue

        df = pd.DataFrame(data, columns=header)
        df.replace(["-", "--", "$ --", "$--", ""], pd.NA, inplace=True)

        for col in df.columns:
            # Convert numeric columns to float64
            if col in NUMERIC_COLS:
                df[col] = pd.to_numeric(
                    df[col]
                    .astype(str)
                    .str.replace("\n", "", regex=False)
                    .str.replace(" ", "", regex=False)
                    .str.replace("$", "", regex=False)
                    .str.replace(",", "", regex=False)
                    .str.replace("(", "-", regex=False)
                    .str.replace(")", "", regex=False),
                    errors="coerce",
                )

            # Convert date columns to datetime
            elif col in DATE_COLS:
                df[col] = pd.to_datetime(df[col], errors="coerce")

        # All other columns remain as 'object' (string) type by default
        final_dfs[name] = df

    return statement_period, final_dfs


def process_all_statements(input_folder, output_file):
    """
    Processes all PDF statements in a folder and saves the aggregated data to a single JSON file.
    """
    all_data = {}
    folder_path = Path(input_folder)
    pdf_files = list(folder_path.glob("*.pdf"))

    print(f"Found {len(pdf_files)} PDF files to process...")

    for pdf_file in pdf_files:
        print(f"Processing {pdf_file.name}...")
        period, dfs = process_single_statement(pdf_file)
        if period:
            # Convert DataFrames to JSON-serializable format (list of dicts)
            # and handle datetime conversion to string format for JSON
            all_data[period] = {
                name: json.loads(df.to_json(orient="records", date_format="iso"))
                for name, df in dfs.items()
            }
        else:
            print(
                f"  - Warning: Could not extract period from {pdf_file.name}. Skipping."
            )

    # Save the master dictionary to a single JSON file
    with open(output_file, "w") as f:
        json.dump(all_data, f, indent=4)

    print(f"\nAll data successfully saved to {output_file}")

## Process Statements & Store as JSON

In [ ]:
process_all_statements(
    input_folder="raw-data/monthly-statements-pdf", output_file="raw-data/brokerage_data.json"
)

Found 29 PDF files to process...
Processing April2025.pdf...
Processing October2023.pdf...
Processing April2024.pdf...
Processing April2023.pdf...
Processing October2024.pdf...
Processing July2023.pdf...
Processing September2023.pdf...
Processing September2024.pdf...
Processing July2024.pdf...
Processing May2023.pdf...
Processing March2023.pdf...
Processing November2024.pdf...
Processing January2024.pdf...
Processing March2024.pdf...
Processing May2024.pdf...
Processing May2025.pdf...
Processing March2025.pdf...
Processing January2025.pdf...
Processing November2023.pdf...
Processing June2025.pdf...
Processing June2024.pdf...
Processing December2023.pdf...
Processing August2023.pdf...
Processing February2023.pdf...
Processing June2023.pdf...
Processing February2024.pdf...
Processing February2025.pdf...
Processing August2024.pdf...
Processing December2024.pdf...

All data successfully saved to brokerage_data.json


## Example Code: Read & Load JSON

In [ ]:
import json
import pandas as pd

with open("raw-data/brokerage_data.json", "r") as f:
    loaded_data = json.load(f)

# Example: Get the Holdings DataFrame for December 2024
pd.DataFrame(loaded_data["2024-December"]["Holdings"])

,Symbol,Description,Quantity,Market Price,Market Value,Cost Price,Unrealized,TD Cost Basis
0,*Cash,USD,NaN,NaN,4.64,NaN,NaN,NaN
1,AMZN,AMAZON COM INC COM,0.149980,219.39,32.90,148.15,10.68,22.22
2,APO,APOLLO GLOBAL MGMT INC COM,0.273329,165.16,45.14,125.12,10.94,34.20
3,ARM,ARM HOLDINGS PLC SPONSORED ADS,0.080826,123.36,9.97,185.58,-5.03,15.00
4,BRW,SABA CAPITAL INCOME & OPRNT FD SHS NEW,4.707718,7.62,35.87,7.43,0.87,35.00
5,CEFS,EXCHANGE LISTED FDS TR SABA INT RATE,1.114267,21.42,23.87,17.95,3.87,20.00
6,CLM,CORNERSTONE STRATEGIC INVESTME COM,2.816941,8.59,24.20,7.10,4.20,20.00
7,GOF,GUGGENHEIM STRATEGIC OPPORTUN COM SBI,3.000000,15.26,45.78,13.42,5.51,40.27
8,GSBD,GOLDMAN SACHS BDC INC SHS,2.436461,12.10,29.48,14.37,-5.52,35.00
9,IBB,ISHARES TR ISHARES BIOTECH,0.047022,132.21,6.22,126.75,0.26,5.96


## Inspect Single Statement Dataframes

In [ ]:
december2024 = process_single_statement("raw-data/monthly-statements-pdf/August2024.pdf")[1]
december2024["Holdings"]

,Symbol,Description,Quantity,Market Price,Market Value,Cost Price,Unrealized,TD Cost Basis
0,*Cash,USD,NaN,NaN,1.01,NaN,NaN,NaN
1,AMZN,AMAZON COM INC COM,0.149980,178.50,26.77,148.15,4.55,22.22
2,ARKK,ARK ETF TR INNOVATION ETF,0.593922,44.82,26.62,48.83,-2.38,29.00
3,ARM,ARM HOLDINGS PLC SPONSORED ADS,0.080826,132.88,10.74,185.58,-4.26,15.00
4,BRW,SABA CAPITAL INCOME & OPRNT FD SHS NEW,4.707718,7.40,34.84,7.43,-0.16,35.00
5,CEFS,EXCHANGE LISTED FDS TR SABA INT RATE,1.114267,21.82,24.31,17.95,4.31,20.00
6,CLM,CORNERSTONE STRATEGIC VALUE FD COM,2.816941,7.59,21.38,7.10,1.38,20.00
7,EWJ,ISHARES INC MSCI JPN ETF NEW,0.194821,71.97,14.02,64.11,1.53,12.49
8,GOF,GUGGENHEIM STRATEGIC OPPORTUN COM SBI,3.000000,15.45,46.35,13.42,6.08,40.27
9,GSBD,GOLDMAN SACHS BDC INC SHS,2.436461,14.13,34.43,14.37,-0.57,35.00
